# Importaciones

In [ ]:
%matplotlib inline
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

Para mostrar los pasos seguidos en el preprocesamiento de imágenes 3d obtenidas con MRI (NIfTI), trabajaremos con el esaceno de un cerebro disponible en la base de datos IXI: https://brain-development.org/ixi-dataset/
(Data License: CC BY-SA 3.0 license, https://creativecommons.org/licenses/by-sa/3.0/)

## Orientación


Primero cargamos la secuencia T1w y extraemos el volumen:

In [ ]:
brain_mri = nib.load("IXI662-Guys-1120-T1.nii.gz")
brain_mri_data = brain_mri.get_fdata()

Luego extraemos su forma y su matriz afín

In [ ]:
shape = brain_mri.shape
affine = brain_mri.affine
print(affine)
print(shape)

In [ ]:
affine[:, 3:]

La matriz afín describe el mapeo de coordenadas pixel a coordenadas del resonador (o del mundo real):
Siempre es es una matriz $4 \times 4$ con el siguiente esquema:

$$
  \left(\begin{array}{cccc}
    \color{red}{x_{1,1}}  & \color{red}{x_{1,2}}  & \color{red}{x_{1,3}} & \color{blue}{x_{1,4}} \\
    \color{red}{x_{2,1}}  & \color{red}{x_{2,2}}  & \color{red}{x_{2,3}} & \color{blue}{x_{1,4}} \\
    \color{red}{x_{3,1}}  & \color{red}{x_{3,2}}  & \color{red}{x_{3,3}} & \color{blue}{x_{1,4}} \\
    \color{red}{0}  & \color{red}{0}  & \color{red}{0} & \color{blue}{1} \\
  \end{array}\right)
$$

La submatriz $\color{red}{\text{roja}}$, $3\times 3$ es responsable de la rotación, escalamiento y cizallamiento de las coordenadas (ver https://es.wikipedia.org/wiki/Transformaci%C3%B3n_af%C3%ADn#Transformaci%C3%B3n_de_im%C3%A1genes).

El vector $\color{blue}{\text{azul}}$ es responsable de la corrección de posición.

La última fila y columna de la matriz se agrega por conveniencia, porque nos permite calcular directamente la transformación entre el sistema de coordenadas en un solo paso en lugar de 2.

podemos acceder al tamaño de voxel, en mm, usando la función ``header.get_zooms()`` provista por **nibabel**.

In [ ]:
print(brain_mri.header.get_zooms())

Puede cortar el volumen en todas las orientaciones: axial, coronal y sagital.<br />
**NOTA: Dependiendo de la orientación del escaneo, los índices cambian**<br />
A veces el primer eje corta axialmente, a veces coronal y a veces sagital.

Puede averiguar la orientación utilizando **nib.aff2axcodes(affine)**
Recuerde la orientación anatómica de la introducción médica.

In [ ]:
nib.aff2axcodes(affine)

En este caso, la orientación del escaneo es:
* **de anterior a posterior (de adelante hacia atrás)**
* **de inferior a superior (de abajo hacia arriba)**
* **de izquierda a derecha**

Las letras devueltas por aff2axcodes siempre indican **el final del eje correspondiente**.

Esto también explica por qué el escaneo está al revés, cuando se corta a lo largo del eje del puño (coronal). A medida que el segundo eje se mueve de abajo hacia arriba, el primer corte **no** es la parte superior de la cabeza, sino una parte del cuello.

Analicemos todas las orientaciones para tener una idea:

Al principio nos movemos a lo largo del primer eje (anterior a posterior). Esto se mueve a través de la cabeza desde la cara hasta la parte posterior de la cabeza.

### Vista coronal

In [ ]:
fig, axis = plt.subplots(1, 2)
axis[0].imshow(brain_mri_data[40, :, :], cmap="gray")
axis[1].imshow(brain_mri_data[120, :, :], cmap="gray")


### Vista axial

In [ ]:
fig, axis = plt.subplots(1, 2)
axis[0].imshow(brain_mri_data[:, 30, :], cmap="gray")
axis[1].imshow(brain_mri_data[:, 200, :], cmap="gray")


### Vista sagital

In [ ]:
fig, axis = plt.subplots(1, 2)
axis[0].imshow(brain_mri_data[:, :, 20], cmap="gray")
axis[1].imshow(brain_mri_data[:, :, 45], cmap="gray")


In [ ]:
fig, axis = plt.subplots(1, 2)

brain_mri_swapped = np.swapaxes(brain_mri_data, 0, 1)
axis[0].imshow(brain_mri_swapped[:, :, 20], cmap="gray")
axis[1].imshow(brain_mri_swapped[:, :, 45], cmap="gray")


## Transformación de coordenadas

Como ejemplo, si desea calcular las coordenadas físicas del desplazamiento, es decir, dónde se encuentran las coordenadas del vóxel (0,0,0) en el espacio físico, simplemente multiplica la matriz afín por esas coordenadas.

Tenga en cuenta que agrega el 1 para mayor comodidad al trabajar con la matriz $4\times 4$

In [ ]:
voxel_coord = np.array((0, 0, 0, 1))
physical_coord0 = affine @ voxel_coord  # @ is a shortcut for matrix multiplication in numpy
print(physical_coord0)

Recordemos que la matriz afín es:

$$\begin{pmatrix}
1.89821944e-02 & -2.72075552e-03 & 1.19975281e+00 & -9.06798553e+01\\
-9.27821696e-01 &  1.32986516e-01 & 2.45456006e-02 & 1.02829445e+02\\
 1.33014351e-01 &  9.28015888e-01 &  5.71511449e-11 & -1.14823784e+02\\
 0.00000000e+00 &  0.00000000e+00 &  0.00000000e+00 & 1.00000000e+00\\
\end{pmatrix}
$$

Al calcular la multiplicación de matrices calculamos efectivamente los siguientes productos:
$$ 1.89e-02 * 0 + -2.7e-03 * 0 + 1.19 * 0 + (-9.06e+01) = -9.06e01 = -90.67$$
$$ -9.27e-01 * 0 +  1.32e-01 * 0 + 2.45e-02 * 0 + 1.028e+02 = 102.8$$
$$  1.33e-01 * 0 +  9.2e-01 * 0 5.71e-11 *0 + (-1.14e+02) = -114.82 $$
$$ 0*0 + 0*0 + 0*0 +1+1 = 1$$

Como puede ver, la incorporación de la cuarta columna y fila habilita este acceso directo.

Por supuesto, también puedes saltarte el atajo y calcular el resultado a mano. Para hacerlo, extrae la submatriz $3\times 3$ y la multiplicas por las coordenadas_voxel. Posteriormente agregas el vector de traducción (sin el último 1)

In [ ]:
voxel_coord_manual = np.array((0, 0, 0))
physical_coord_manual = affine[:3, :3] @ voxel_coord_manual
physical_coord_manual = physical_coord_manual + affine[:3,3]
print(physical_coord_manual)

Si desea transformar coordenadas físicas en coordenadas de píxeles/vóxeles, debe calcular la inversa de la matriz afín (``np.linalg.inv(arr)`` y luego multiplicar esta inversa con las coordenadas físicas.
Como ejemplo, si queremos obtener las coordenadas de vóxel de nuestro desplazamiento<br /> ( -90.67985535 102.82944489 -114.8237838
) necesitamos realizar el siguiente cálculo:

In [ ]:
voxel_coords = (np.linalg.inv(affine) @ physical_coord0).round()
print(voxel_coords)

## Reorientación

Si lo desea, puede reorientar el volumen a RAS usando ``nibabel.as_closest_canonical(nifti)``<br />
Esto también se llama orientación canónica.

In [ ]:
brain_mri_canonical = nib.as_closest_canonical(brain_mri)
brain_mri_canonical_data = brain_mri_canonical.get_fdata()

In [ ]:
canonical_affine = brain_mri_canonical.affine
print(affine)
print("----")
print(canonical_affine)
print(nib.aff2axcodes(canonical_affine))

¿Puedes ver las diferencias? simplemente permutó las entradas en la submatriz $3 \times 3$.
El desplazamiento, por supuesto, cambia porque ahora comenzamos en posiciones diferentes.

In [ ]:
fig, axis = plt.subplots(1, 2)
axis[0].imshow(brain_mri_canonical_data[50, :, :], cmap="gray")
axis[1].imshow(brain_mri_canonical_data[130, :, :], cmap="gray")


In [ ]:
fig, axis = plt.subplots(1, 2)
axis[0].imshow(brain_mri_canonical_data[:, 40, :], cmap="gray")
axis[1].imshow(brain_mri_canonical_data[:, 90, :], cmap="gray")


In [ ]:
fig, axis = plt.subplots(1, 2)
axis[0].imshow(brain_mri_canonical_data[:, :, 5], cmap="gray")
axis[1].imshow(brain_mri_canonical_data[:, :, 70], cmap="gray")


# Remuestreo

Es posible que también desee cambiar el tamaño de su escaneo, ya que podría ser demasiado grande para su sistema.
Sin embargo, cambiar el tamaño de un volumen no es tan fácil como cambiar el tamaño de una imagen porque es necesario cambiar el tamaño del vóxel.

Cambiemos el tamaño de nuestra resonancia magnética cerebral de (256, 256, 150) a (128, 128, 100)

In [ ]:
print(brain_mri.shape)
print(brain_mri.header.get_zooms())

Para hacerlo, puede usar ``conform(input, wanted_shape, voxel_size)`` de *nibabel.processing*, que vuelve a muestrear la imagen con la forma deseada.
Tenga en cuenta que debe cambiar el tamaño del vóxel, ya que de lo contrario sería imposible reducir el tamaño.
Simplemente usemos un tamaño de vóxel de $(2 \times 2 \times 2)$mm

In [ ]:
import nibabel.processing

In [ ]:
voxel_size = (2, 2, 2)
brain_mri_resized = nibabel.processing.conform(brain_mri, (128, 128, 100), voxel_size, orientation="PSR")
brain_mri_resized_data = brain_mri_resized.get_fdata()

In [ ]:
print(brain_mri.shape)
print(brain_mri_resized.shape)
print(brain_mri_resized.header.get_zooms())

Como puede ver, ¡la imagen remuestreada todavía se parece a la original!

In [ ]:
IDX = 50
fig, axis = plt.subplots(1, 2)
axis[0].imshow(brain_mri_data[:,:,IDX], cmap="gray")
axis[1].imshow(brain_mri_resized_data[:,:,IDX], cmap="gray")


## Normalización y estandarización

## CT

Como las TCs tienen una escala fija de -1000 (aire) a 1000 (agua) (esto se llama unidad de Houndsfield), normalmente no realiza la normalización para mantener esas escalas.

En la práctica, se puede suponer que los valores están entre -1024 y 3071.

De esta manera puedes estandarizar los datos dividiendo el volumen por 3071.

Para demostrar esto, cargaremos la TC de un sujeto contenido en el desafío de segmentación de tumores de pulmón decathlon de segmentación médica (http://medicaldecathlon.com/, https://arxiv.org/abs/1902.ctorio (Licencia: CC BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/)

In [ ]:
lung_ct = nib.load("lung_043.nii.gz")
lung_ct_data = lung_ct.get_fdata()

In [ ]:
lung_ct_data_standardized = lung_ct_data / 3071

In [ ]:
plt.figure()
plt.imshow(np.rot90(lung_ct_data_standardized[:,:,50]), cmap="gray")

## Enventanado:

Dependiendo de la tarea que realices querrás tener un contraste diferente.<br />
Por ejemplo, si desea inspeccionar el pulmón, es importante poder ver todos los vasos pequeños.<br />
En cambio, a la hora de examinar el cuerpo es importante tener un alto contraste, para poder diferenciar distintos tejidos.

Este cambio en contraste se llama ventana.<br />
Generalmente hay cuatro ventanas diferentes: una ventana de pulmón, una ventana de hueso, una ventana de tejido blando y una ventana de cerebro.<br />
Puede crear una ventana de este tipo recortando todos los valores de píxeles mayores que un umbral.

Echemos un vistazo a la ventana de los pulmones y los tejidos blandos.

Para obtener una buena ventana pulmonar, puede recortar todos los valores mayores que -1000 a -500. Tenga en cuenta que esta ventana nos impide por completo echar un vistazo al abdomen ya que todo parece idéntico.

In [ ]:
lung_ct_lung_window = np.clip(lung_ct_data, -1000, -500)

fig, axis = plt.subplots(1, 2)
axis[0].imshow(np.rot90(lung_ct_lung_window[:,:,50]), cmap="gray")
axis[1].imshow(np.rot90(lung_ct_lung_window[:,:,5]), cmap="gray")
axis[0].axis("off")
axis[1].axis("off")
fig.suptitle("Lung Window")
plt.tight_layout()
plt.savefig("lung_window.png", bbox_inches="tight")


Para obtener una buena ventana de tejido blando, puede recortar todos los valores entre -250 y 250. Aquí el pulmón es casi negro pero tiene una bonita imagen del abdomen.

In [ ]:
lung_ct_soft_tissue_window = np.clip(lung_ct_data, -250, 250)

fig, axis = plt.subplots(1, 2)
axis[0].imshow(np.rot90(lung_ct_soft_tissue_window[:,:,50]), cmap="gray")
axis[1].imshow(np.rot90(lung_ct_soft_tissue_window[:,:,5]), cmap="gray")

axis[0].axis("off")
axis[1].axis("off")
fig.suptitle("Soft Tissue Window")
plt.tight_layout()
plt.savefig("tissue_window.png", bbox_inches="tight")


## MRI
A diferencia de las tomografías computarizadas, las imágenes de resonancia magnética no tienen una escala fija absoluta y cada paciente varía.

Por lo tanto, puede normalizar $z$ las exploraciones según el paciente.

$$ X_{\text{norm}} = \frac{X - \mu}{\sigma}$$

Además, puede realizar un escalado mínimo-máximo.
$$X_{\text{standardized}} = \frac{X - \mathrm{min}(X)}{\mathrm{max}(X) - \mathrm{min}(X)} $$

Para demostrar esto, utilizaremos una resonancia magnética cardíaca de la tarea de segmentación de aurículas de decatlón de segmentación médica. (Los mismos enlaces que para el CT)

In [ ]:
cardiac_mri = nib.load("la_003.nii.gz")
cardiac_mri_data = cardiac_mri.get_fdata()

In [ ]:
mean, std = np.mean(cardiac_mri_data), np.std(cardiac_mri_data)
cardiac_mri_norm = (cardiac_mri_data - mean) / std
cardiac_mri_standardized = (cardiac_mri_norm - np.min(cardiac_mri_norm)) / (np.max(cardiac_mri_norm) - np.min(cardiac_mri_norm))

In [ ]:
plt.figure()
plt.imshow(cardiac_mri_standardized[:,:,30], cmap="gray")

Normalmente no hay ventanas en las exploraciones por resonancia magnética.
Esas dos técnicas de normalización deberían adaptarse a la mayoría de las tareas.
Tenga en cuenta que existen algunas técnicas más específicas, como la normalización de la materia blanca para resonancias magnéticas cerebrales o la corrección de campo de sesgo para imágenes de resonancia magnética ligeramente corruptas.